In [1]:
def parse_results_num(file, version):
    with open(file) as f:
        lines = f.readlines() 
        num_covered_targets = 0
        num_potential_faults = 0
        num_potential_500_faults = 0
        
        # if version == 'v200':
        #     pt500_text_to_check = "* Potential 500 faults IDs: "
        # else:
        #     pt500_text_to_check = "* Potential 500 faults: "

        for line in lines:
            if "* Covered targets (lines" in line:
                num_covered_targets = int(line.split("etc.): ")[1].split('\n')[0])
            if "* Potential faults: " in line:
                num_potential_faults = int(line.split('faults: ')[1].split('\n')[0])
            if "* Potential 500 faults: " in line:
                num_potential_500_faults = int(line.split("* Potential 500 faults: ")[1].split('\n')[0])

        if num_covered_targets == 0 and num_potential_faults == 0 and num_potential_500_faults == 0:
            print(f"Results missing for file: {file}")
        
        return num_covered_targets, num_potential_faults, num_potential_500_faults
       


In [6]:
num_iterations = 11
app = "catwatch"
algorithms = ['MOSA', 'MISH', 'MISHMOSA']
fitness_function = 'weighted_size'
# if algorithm == 'MOSA':
result_format = f'with_500_faults_EM_logs'
version = 'v200'
# else:
    # result_format = f'with_500_faults_{fitness_function}_EM_logs'

all_runs_covered_targets = dict()
all_runs_potential_faults = dict()
all_runs_potential_500_faults = dict()

for algorithm in algorithms:
    covered_targets = []
    potential_faults = []
    potential_500_faults = []
    for i in range(1, num_iterations):
        file = f"../../results_{app}_{version}_{fitness_function}_latest/{app}_{result_format}_{algorithm}_{i}.txt"
        ct, pt, pt500 = parse_results_num(file, version)
        covered_targets.append(ct)
        potential_faults.append(pt)
        potential_500_faults.append(pt500)
    
    all_runs_covered_targets[algorithm] = covered_targets
    all_runs_potential_faults[algorithm] = potential_faults
    all_runs_potential_500_faults[algorithm] = potential_500_faults


In [7]:
# MOSA_covered_targets = all_runs_covered_targets['MOSA']
MISH_covered_targets = all_runs_covered_targets['MISH']
MISHMOSA_covered_targets = all_runs_covered_targets['MISHMOSA']

# Print them in table format
for i in range(num_iterations - 1):
    # print(f"{MOSA_covered_targets[i]} \t {MISH_covered_targets[i]} \t {MISHMOSA_covered_targets[i]}")
    print(f"{MISH_covered_targets[i]} \t {MISHMOSA_covered_targets[i]}")

15543 	 12599
2911 	 20688
3049 	 3109
2940 	 2883
2965 	 2986
3040 	 10558
10824 	 2965
3026 	 18546
2992 	 2971
3081 	 3065


In [8]:
# MOSA_potential_faults = all_runs_potential_faults['MOSA']
MISH_potential_faults = all_runs_potential_faults['MISH']
MISHMOSA_potential_faults = all_runs_potential_faults['MISHMOSA']

for i in range(num_iterations - 1):
    # print(f"{MOSA_potential_faults[i]} \t {MISH_potential_faults[i]} \t {MISHMOSA_potential_faults[i]}")
    print(f"{MISH_potential_faults[i]} \t {MISHMOSA_potential_faults[i]}")

7 	 9
6 	 6
6 	 6
7 	 7
7 	 8
6 	 7
7 	 6
6 	 10
7 	 7
6 	 6


In [10]:
# MOSA_potential_500_faults = all_runs_potential_500_faults['MOSA']
MISH_potential_500_faults = all_runs_potential_500_faults['MISH']
MISHMOSA_potential_500_faults = all_runs_potential_500_faults['MISHMOSA']

for i in range(num_iterations - 1):
    # print(f"{MOSA_potential_500_faults[i]} \t {MISH_potential_500_faults[i]} \t {MISHMOSA_potential_500_faults[i]}")
    print(f"{MISH_potential_500_faults[i]} \t {MISHMOSA_potential_500_faults[i]}")

4 	 6
4 	 3
4 	 4
5 	 5
5 	 6
4 	 4
4 	 4
4 	 7
5 	 5
4 	 4


# Computing statistics for paper.

In [31]:
from statistics import median
num_iterations = 11
applications = ['catwatch', 'features-service', 'scout-api', 'proxyprint', 'languagetool']
algorithms = ['MOSA', 'MISH', 'MISHMOSA']
fitness_functions = ['lower_median', 'weighted_size']
abbrev_map = {'lower_median': 'LM', 'weighted_size': 'WS'}
result_format = f'with_500_faults_EM_logs'
version = 'v200'

app_results = dict()
for app in applications:
    app_results[app] = dict()
    all_runs_covered_targets = dict()
    all_runs_potential_faults = dict()
    all_runs_potential_500_faults = dict()

    for fitness_function in fitness_functions:
        for algorithm in algorithms:
            if algorithm == 'MOSA' and fitness_function == 'weighted_size':
                continue

            covered_targets = []
            potential_faults = []
            potential_500_faults = []
            for i in range(1, num_iterations):
                file = f"../../results_{app}_{version}_{fitness_function}_latest/{app}_{result_format}_{algorithm}_{i}.txt"
                ct, pt, pt500 = parse_results_num(file, version)
                covered_targets.append(ct)
                potential_faults.append(pt)
                potential_500_faults.append(pt500)
            
            if algorithm == 'MOSA':
                method_name = algorithm
            else:
                method_name = f"{algorithm} - {abbrev_map[fitness_function]}"

            all_runs_covered_targets[method_name] = covered_targets
            all_runs_potential_faults[method_name] = potential_faults
            all_runs_potential_500_faults[method_name] = potential_500_faults

    app_results[app]['covered_targets'] = all_runs_covered_targets
    app_results[app]['potential_faults'] = all_runs_potential_faults
    app_results[app]['potential_500_faults'] = all_runs_potential_500_faults

# first print the method names in one line
method_names = ['MOSA', 'MISH - LM', 'MISH - WS', 'MISHMOSA - LM', 'MISHMOSA - WS']
print('\t'.join(method_names))

# Then print the median covered targets
all_medians = []

for app in applications:
    medians = []
    for method_name in method_names:
        medians.append(str(median(app_results[app]['potential_500_faults'][method_name])))

    all_medians.append(medians)

for medians in all_medians:
    print('\t'.join(medians))






MOSA	MISH - LM	MISH - WS	MISHMOSA - LM	MISHMOSA - WS
10.0	7.0	7.0	8.0	7.5
25.0	24.0	23.5	25.0	24.5
57.5	52.5	45.0	54.5	50.5
29.0	34.0	34.5	34.0	34.0
5.0	5.5	4.0	5.0	4.5
